# Erste Schritte

Zunächst feststellen wo wir sind:

In [1]:
pwd

/home/vagrant/ais/hands-on-ansible/00-home-base


Und welche Dateien es dort gibt:

In [2]:
ls -lachi

insgesamt 52K
5512662 drwxrwxr-x 3 vagrant vagrant 4,0K Sep 30 13:10 .
5512620 drwxrwxr-x 8 vagrant vagrant 4,0K Sep 29 09:17 ..
5512663 -rw-rw-r-- 1 vagrant vagrant   60 Sep 29 09:17 2.1_creating_Ansible_Home_Base.sh
5512664 -rw-rw-r-- 1 vagrant vagrant  359 Sep 29 09:17 2.2_Ansible_Bleeding_Edge.sh
5512665 -rw-rw-r-- 1 vagrant vagrant  498 Sep 29 09:17 2.3_Ansible_pip.sh
5512666 -rwxrwxr-x 1 vagrant vagrant  737 Sep 30 07:17 2.4_lxc_setup.sh
5512667 -rwxrwxr-x 1 vagrant vagrant 1,1K Sep 30 12:19 ais-lxc-setup.sh
5505067 -rw-r--r-- 1 vagrant vagrant  15K Sep 30 13:10 Erste_Schritte_LXC.ipynb
5641189 drwxr-xr-x 2 vagrant vagrant 4,0K Sep 30 07:46 .ipynb_checkpoints
5512760 -rw-r--r-- 1 vagrant vagrant  278 Sep 30 12:57 setup-lxc.yml


Der Inhalt der Datei ais-lxc-setup.sh

In [3]:
cat ais-lxc-setup.sh

#!/bin/bash

-set x

# Path for prepare playbook
PREPARE=~/ais/hands-on-ansible/02-playbooks/00-simple-playbook-examples/prepare_ansible_target.yml
MAKE_SSH_KEY="ssh-keygen -b 2048 -t rsa -f ~/.ssh/id_rsa -q -N "" creates=~/.ssh/id_rsa"

# Cnerate containers
for i in ais-bashy web1 web2 db playbooks; do
	sudo lxc-create -n ${i} -t ubuntu && \
	# Remove legacy configuration keys \
	sudo lxc-update-config -c /var/lib/lxc/${i}/config && \
	# Start container \ 
	sudo lxc-start -n ${i} && \
	# Prepare ais-bashy \
	if [[ ${i} == "ais-bashy" ]]; then
		echo "Wait for comming up"
		sleep 5
		# Get ip
		ip=$(sudo lxc-ls --fancy | grep ${i} | cut -d '-' -f 3 | tr -d ' ')
		# Remove host key for ip
		ssh-keygen -R ${ip}
		# Add ip into known_hosts
		ssh-keyscan -H ${ip} >> ~/.ssh/known_hosts
		# Setup ssh-key for user
		ansible -i localhost, -m shell -a "MAKE_SSH_KEY" --connection=local localhost
		# Run prepare playbook
		ansible-playbook -i ${ip}, -u ubuntu --ask-pass --ask-become-pass ${PREPAR

Dieses Skript ist inteaktiv, es frägt nach dem Loginpasswort `ubuntu` und suod-Passwort `ubuntu`.

Mit `expect` bzw. mit dem python-Modul pexpect können diese Fragen per Skript beantwortet werden.

Für `expect` gibt es das gleichnamige ansible-Modul. Die Dokumentation findet man hier:

http://docs.ansible.com/ansible/latest/expect_module.html#requirements-on-host-that-executes-module

Es werden die Abhängigkeiten installiert:

In [4]:
sudo apt-get update && sudo apt-get install --yes python-pexpect python3-pexpect

Ign:1 http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 InRelease
Ign:2 http://dl.google.com/linux/chrome/deb stable InRelease                   
Holen:3 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]   
OK:4 http://ppa.launchpad.net/ansible/ansible/ubuntu xenial InRelease          
OK:5 http://repo.mongodb.org/apt/ubuntu xenial/mongodb-org/3.4 Release         
OK:6 http://ppa.launchpad.net/gnome-terminator/ppa/ubuntu xenial InRelease     
OK:7 http://dl.google.com/linux/chrome/deb stable Release                      
OK:8 http://ppa.launchpad.net/webupd8team/atom/ubuntu xenial InRelease         
OK:9 http://us.archive.ubuntu.com/ubuntu xenial InRelease                      
OK:10 https://deb.nodesource.com/node_6.x xenial InRelease
OK:12 http://us.archive.ubuntu.com/ubuntu xenial-updates InRelease
Holen:13 http://us.archive.ubuntu.com/ubuntu xenial-backports InRelease [102 kB]
Es wurden 204 kB in 1 s geholt (193 kB/s).                     
Paketlisten wer

Somit kann das ansible-Module `expect` verwendet werden.

In [5]:
cat setup-lxc.yml

---
- hosts: all
  connection: local
  gather_facts: no

  tasks:
  - name: Init AIS lxc containers
    expect:
      command: /home/vagrant/ais/hands-on-ansible/00-home-base/ais-lxc-setup.sh
      responses:
        (?i)password: "ubuntu"
      creates: /var/lib/lxc/ais-bashy


Unser Skript wird aufgerufen und falls der String `password` (case insensitive) erscheint, wird `ubuntu` als Passwort eingegeben.

In [6]:
ansible-playbook -i localhost, setup-lxc.yml

 ____________
< PLAY [all] >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

 ________________________________
< TASK [Init AIS lxc containers] >
 --------------------------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

changed: [localhost]
 ____________
< PLAY RECAP >
 ------------
        \   ^__^
         \  (oo)\_______
            (__)\       )\/\
                ||----w |
                ||     ||

localhost                  : ok=1    changed=1    unreachable=0    failed=0   



Wir können nun die Container anzeigen lassen.

In [7]:
sudo lxc-ls

ais-bashy db        playbooks web1      web2      


In [8]:
sudo lxc-ls --help

Usage: lxc-ls 
[-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]
[-1] [-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]
[-f] [-P lxcpath] [--active] [--running] [--frozen] [--stopped] [--nesting] [-g groups] [--filter regex]

lxc-ls list containers

Options :
  -1, --line         show one entry per line
  -f, --fancy        use a fancy, column-based output
  -F, --fancy-format comma separated list of columns to show in the fancy output
                     valid columns are: NAME, STATE, PID, RAM, SWAP, AUTOSTART,
                     GROUPS, INTERFACE, IPV4 and IPV6
  --active           list only active containers
  --running          list only running containers
  --frozen           list only frozen containers
  --stopped          list only stopped containers
  --defined          list only defined containers
  --nesting=NUM      list nested containers up to NUM (default is 5) levels of

In [9]:
sudo lxc-ls --fancy

NAME      STATE   AUTOSTART GROUPS IPV4       IPV6 
ais-bashy RUNNING 0         -      10.0.3.200 -    
db        RUNNING 0         -      10.0.3.198 -    
playbooks RUNNING 0         -      10.0.3.210 -    
web1      RUNNING 0         -      10.0.3.56  -    
web2      RUNNING 0         -      10.0.3.130 -    


Nur für unseren ais-bashy lautet das Kommando:

In [10]:
sudo lxc-ls ais-bashy --fancy

NAME      STATE   AUTOSTART GROUPS IPV4       IPV6 
ais-bashy RUNNING 0         -      10.0.3.200 -    


In [11]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $1 }'

NAME
ais-bashy


In [12]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $2 }'

STATE
RUNNING


In [13]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $3 }'

AUTOSTART
0


In [14]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $4 }'

GROUPS
-


In [15]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }'

IPV4
10.0.3.200


In [16]:
sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.'

10.0.3.200


In [17]:
export AIS_BASHY_IP=$(sudo lxc-ls ais-bashy --fancy | awk '{ print $5 }' | grep '.\.')
echo $AIS_BASHY_IP

10.0.3.200


## Uptime via ssh und ansible erfragen

In [18]:
ssh root@${AIS_BASHY_IP} uptime

 13:17:19 up  6:25,  0 users,  load average: 0,08, 0,16, 0,17


In [19]:
ls ~/.ssh

authorized_keys  config  id_rsa  id_rsa.pub  known_hosts


In [20]:
type echo

echo ist eine von der Shell mitgelieferte Funktion.


In [21]:
help echo

echo: echo [-neE] [Argument ...]
    Ausgabe der Argumente auf die Standardausgabe.
    
    Zeigt die Argumente auf der Standardausgabe gefolgt von einem
    Zeilenumbruch an.
    
    Optionen:
      -n	Keinen Zeilenumbruch anfügen
      -e	Interpretation der folgenden Escape-Sequenzen zulassen
      -E	Keine Interpretation der Escape-Sequenzen.
    
    »echo« interpretiert die folgenden Escape-Sequenzen:
      \a	Alarm (Glocke)
      \b	Rücktaste (Backspace)
      \c	weitere Ausgabe unterdrücken
      \e	Escape-Zeichen
      \E        Escape Zeichen
      \f	Seitenvorschub
      \n	Zeilenvorschub
      \r	Wagenrücklauf
      \t	Horizontaler Tabulator
      \v	Vertikaler Tabulator
      \\        umgekehrter Schrägstrich (Backslash)
      \0nnn	Zeichen mit dem ASCII-Code »NNN« (oktal). »NNN« kann null
    		bis drei oktale Ziffern haben.
      \xHH	Acht-Bit-Zeichen mit dem Wert »HH« (hexadezimal). »HH«
    		kann ein oder zwei hexadezimale Ziffern haben.
    
    Rückgabewert:
    G

Achtung dies überschreibt eine bestehende ssh-config Datei.

In [22]:
echo -e "HOST AIS-BASHY\n\tHostName\t${AIS_BASHY_IP}\n\tUser\troot" > ~/.ssh/config

In [23]:
cat ~/.ssh/config

HOST AIS-BASHY
	HostName	10.0.3.200
	User	root


In [24]:
ssh AIS-BASHY uptime

 13:18:08 up  6:26,  0 users,  load average: 0,04, 0,14, 0,16


In [25]:
ansible -a uptime -i ${AIS_BASHY_IP}, ${AIS_BASHY_IP} -u root

10.0.3.200 | SUCCESS | rc=0 >>
 13:18:15 up  6:26,  1 user,  load average: 0,03, 0,13, 0,16


## Aufräumen

Achtung dies löscht alle Container ohne wenn und aber.

In [26]:
for i in ais-bashy db web1 web2 playbooks; do
    sudo lxc-destroy --force --name ${i}
done

Destroyed container ais-bashy
Destroyed container db
Destroyed container web1
Destroyed container web2
Destroyed container playbooks


In [27]:
sudo lxc-ls --fancy